In [28]:
# Import packages
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import os
import json

In [32]:
# Global inputs
timestamps = 'TIMESTAMP("2023-09-25")'

In [33]:
# First, set the credentials
key_path_home_dir = os.path.expanduser("~") + "/bq_credentials.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path_home_dir, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

# Now, instantiate the client and upload the table to BigQuery
client = bigquery.Client(project="web-scraping-371310", credentials=credentials)

# Query the NULL records in the "lukas_mobile_de" table
query = f"""
    SELECT *
    FROM `web-scraping-371310.crawled_datasets.lukas_mobile_de`
    WHERE TIMESTAMP_TRUNC(crawled_timestamp, DAY) IN ({timestamps}) AND titel IS NULL
"""

# Execute the query
df_null_records = pd.DataFrame(client.query(query=query).result().to_dataframe())

# Sort the dataframe by the crawled_timestamp, marke, modell
df_null_records = df_null_records.sort_values(
    by=["crawled_timestamp", "marke", "modell"]
)

In [34]:
# Create a list of the URLs to be crawled in the correct JSON format
final_urls = []
for m in df_null_records["marke"].unique():
    df_iter = df_null_records[df_null_records["marke"] == m].reset_index(drop=True)
    urls_iter = []
    for i in range(len(df_iter)):
        output_dict = {
            "marke": df_iter["marke"][i],
            "modell": df_iter["modell"][i],
            "last_page": int(df_iter["total_num_pages"][i]),
            "page_rank": int(df_iter["page_rank"][i]),
            "car_page_url": df_iter["url_to_crawl"][i],
        }
        urls_iter.append(output_dict)
    final_urls.append(urls_iter)
    
# Write the results to a JSON file
with open(f"car_page_url_list_cat_all.json", mode="w", encoding="utf-8") as f:
    json.dump(obj=final_urls, fp=f, ensure_ascii=False, indent=4)